In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('train.csv')

In [3]:
df=df.dropna()
df = df.reset_index()
x = df.drop('label',axis=1)
y = df['label']
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [4]:
messages = df['title']
y =df['label']
import re
corpus1 = []
for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages[i])
    review = review.lower()
    review = review.split()
    review = ' '.join(review)
    corpus1.append(review)
len(corpus1)

18285

In [5]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [6]:
corpus = []
for d in corpus1:
    doc1 = nlp(d)
    word = [w.lemma_ for w in doc1 if nlp.vocab[str(w)].is_stop==False]
    corpus.append(' '.join(word))
corpus[0]

'house dem aide didn t comey s letter jason chaffetz tweet'

In [22]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from keras.layers import Dropout

In [23]:
voc_size=5000
one_hotrep = [one_hot(words,voc_size) for words in corpus]

In [24]:

sent_length=25
embedded_docs=pad_sequences(one_hotrep,padding='pre',maxlen=sent_length)
print(type(embedded_docs[0]))

<class 'numpy.ndarray'>


In [25]:
X = embedded_docs
y = np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [30]:
## Creating model
embedding_vector_features=40
regressor=Sequential()
regressor.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
regressor.add(LSTM(200,return_sequences=True))    
regressor.add(Dropout(0.2))
regressor.add(LSTM(100))   
regressor.add(Dropout(0.2)) 
regressor.add(Dense(1,activation='sigmoid'))
regressor.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics=['accuracy'])
print(regressor.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 25, 40)            200000    
                                                                 
 lstm_4 (LSTM)               (None, 25, 200)           192800    
                                                                 
 dropout_2 (Dropout)         (None, 25, 200)           0         
                                                                 
 lstm_5 (LSTM)               (None, 100)               120400    
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 513,301
Trainable params: 513,301
Non-tr

In [31]:
regressor.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 27s 117ms/step - loss: 0.1062 - accuracy: 0.8466 - val_loss: 0.0575 - val_accuracy: 0.9253
Epoch 2/10
192/192 [==============================] - 22s 116ms/step - loss: 0.0386 - accuracy: 0.9514 - val_loss: 0.0611 - val_accuracy: 0.9218
Epoch 3/10
192/192 [==============================] - 23s 121ms/step - loss: 0.0263 - accuracy: 0.9687 - val_loss: 0.0705 - val_accuracy: 0.9070
Epoch 4/10
192/192 [==============================] - 23s 119ms/step - loss: 0.0192 - accuracy: 0.9769 - val_loss: 0.0720 - val_accuracy: 0.9117
Epoch 5/10
192/192 [==============================] - 22s 117ms/step - loss: 0.0139 - accuracy: 0.9841 - val_loss: 0.0716 - val_accuracy: 0.9163
Epoch 6/10
192/192 [==============================] - 21s 112ms/step - loss: 0.0105 - accuracy: 0.9885 - val_loss: 0.0728 - val_accuracy: 0.9158
Epoch 7/10
192/192 [==============================] - 22s 115ms/step - loss: 0.0098 - accuracy: 0.9893 - val_loss: 0.0785 - val_ac